In [ ]:
#| default_exp cern.converter

# CERN Converter
> adds clustering and charge to trees created with pXar

In [ ]:
#| export
#!/usr/bin/env python

In [ ]:
#| export
# created on August 30th 2018 by M. Reichmann (remichae@phys.ethz.ch)
# --------------------------------------------------------
from subprocess import check_call
import uproot
from fastcore.script import *

from HighResAnalysis.plotting.utils import array, info as pinfo
from HighResAnalysis.src.converter import Converter, DUT
from HighResAnalysis.plotting.utils import download_file, remove_file, warning
from HighResAnalysis.src.analysis import Analysis

Welcome to JupyROOT 6.26/10


In [ ]:
#| export
class CERNConverter(Converter):
    """
    Converts the raw data taken at CERN in several steps to hdf5 files.
    The raw data consists of one root file for the DUTs saved with pXar and a binary file from the KARTEL telescope.
    STEP -2: raw -> root for tel          (judith)
    STEP -1: adc -> vcal for DUTs         (python)
    STEP  0: merge tel and dut root files (python)\n"""
    __doc__ += '\n'.join(Converter.__doc__.split('\n')[3:])

    def __init__(self, data_dir, run_number, dut_name=None):

        Converter.__init__(self, data_dir, run_number, dut_name)

        self.Adc2Vcal = self.init_adc2vcal()
        self.EventAlignment = self.init_event_alignment()
        self.Ref = self.init_ref()

    def proteus_raw_file_path(self):
        return self.DataDir.joinpath('data', f'run{self.Run:04d}.root')

    def trigger_info_file(self):
        return self.Adc2Vcal.OutFilePath

    @property
    def first_steps(self):
        return self.Raw.Steps + self.Adc2Vcal.Steps + self.Ref.Steps + [(self.merge_root_files, self.proteus_raw_file_path())]

    @property
    def aux_files(self):
        return [self.Raw.OutFilePath, self.Adc2Vcal.OutFilePath, self.Ref.OutFilePath] + super().aux_files

    @property
    def raw_files(self):
        return [self.EventAlignment.RefPath, self.Adc2Vcal.RawFilePath] + super().raw_files

    def init_raw(self):
        from cern.raw import CERNRaw
        return CERNRaw(self)

    def init_event_alignment(self):
        from cern.event_alignment import EventAlignment
        return EventAlignment(self.Raw)

    def init_adc2vcal(self):
        from cern.adc import Adc2Vcal
        return Adc2Vcal(self)

    def init_ref(self):
        from cern.ref import RefConverter
        return RefConverter(self)

    def init_duts(self):
        return super().init_duts() if self.DUTName is None else [DUT.from_name(self.DUTName, self.Run.Info)]

    def merge_root_files(self, force=False):
        """merge the telescope and DUT root files"""
        self.OutFilePath.parent.mkdir(exist_ok=True)
        cmd = f'hadd{" -f" if force else ""} {self.proteus_raw_file_path()} {self.Raw.OutFilePath} {self.Ref.OutFilePath} {self.Adc2Vcal.OutFilePath}'
        pinfo(cmd)
        check_call(cmd, shell=True)

    @property
    def time_stamp_file(self):
        return self.Adc2Vcal.OutFilePath

    def get_time_stamp(self):
        return (array(uproot.open(self.time_stamp_file)['Event']['TimeStamp']) / 1000).astype('d')

    @property
    def calibration(self):
        from cern.calibration import CERNCalibration
        return CERNCalibration

In [ ]:
#| export
@call_parse
def main(run:int=232,
        dut:int=0,
        tc:str=None):
    # from argparse import ArgumentParser
    # parser = ArgumentParser()
    # parser.add_argument('run', nargs='?', default=232)
    # parser.add_argument('dut', nargs='?', default=0)
    # parser.add_argument('-tc', nargs='?', default=None)
    # pargs = parser.parse_args()

    ana_ = Analysis(tc)
    tc = ana_.BeamTest

    z = (Converter if tc.Location == 'DESY' else CERNConverter).from_ana(run, dut, ana_)
    r = z.Raw
    p = z.Proteus
    c = z.load_calibration()
    rn = z.Run
    draw = c.Draw
    if hasattr(z, 'Adc2Vcal'):
        adc = z.Adc2Vcal

In [ ]:
#| hide
from nbdev import *
nbdev_export()